In [2]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
db_conn = db.get_connection()
video_id = "7520882134135868695"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [4]:
video_path = os.path.abspath("./tmp/video/7520882134135868695.mp4")
audio_path = os.path.abspath("./tmp/video/7520882134135868695.wav")

In [5]:
video_url, video_path, audio_path = download.tiktok_full(video_id)


INFO - [EnrichmentWorker] - Downloading MP4 for 7520882134135868695 → /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520882134135868695.mp4
INFO - [EnrichmentWorker] - Successfully extracted audio to: /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7520882134135868695.wav


In [4]:
# model_name = "ggml-large-v3.bin" 
# transcript, lang, output = transcribe_audio(audio_path, model_name)
# logging.info(transcript)

INFO - [EnrichmentWorker] - Starting transcriber model ggml-large-v3.bin for audio in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7520882134135868695.wav
INFO - [EnrichmentWorker] - Călin Georgescu este născut în ziua de 26 sub arhetipul împăratului.
 Nu este întâmplător că el vorbește despre ordine și adevăr.
 Este un destin care cere viziune, putere și disciplină.
 Dar în umbră apare conflictul cu autoritățile și lupta între adevăr și control.
 Tu ce crezi? Călin Georgescu este un lider sau un luptător cu sistemul?
 Dacă vrei să afli ziua ta de naștere, intră pe site-ul meu www.ralucaleahu.ro unde găsești karma zilei tale de naștere.


In [5]:
# sentiment_results = SentimentAnalyzer.get_sentiment(transcript, "ro")
# sentiment_results

INFO - [EnrichmentWorker] - Loading model DGurgurov/xlm-r_romanian_sentiment for language ro
INFO - [EnrichmentWorker] - Successfully loaded model DGurgurov/xlm-r_romanian_sentiment
/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
INFO - [EnrichmentWorker] - SENTIMENT MODEL OUTPUT SequenceClassifierOutput(loss=None, logits=tensor([[0.3686, 0.0584]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INFO - [EnrichmentWorker] - SENTIMENT MODEL SCORES [0.57691735 0.42308265]


In [7]:
# description = get_video_description(video_id, db_conn)
# description

'🎯 Călin Georgescu s-a născut sub arhetipul Împăratului. O zi care cere ordine, autoritate, viziune. Dar și o karmă grea: să nu abuzezi de putere și să-ți supui ego-ul Adevărului. 👁\u200d🗨 În umbră, apare lupta cu autoritățile și conflictul dintre vocea interioară și sistem. ❓Tu ce vezi în el? Un lider? Sau un luptător cu sistemul? 👇 Lasă-mi părerea ta. 🔗 Și intră pe  https://www.ralucaleahu.ro/  să descoperi karma zilei tale de naștere. #ralucaleahu #calingeorgescupresedinte #suveranitate #foryiurpage #tiktokromania🇷🇴 #tarot '

In [8]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520882134135868695.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video discusses Călin Georgescu, stating he was born under the archetype of 'The Emperor,' which signifies a destiny requiring vision, power, and discipline, but also carries a karma of not abusing power and submitting ego to truth. The speaker claims that Georgescu's focus on order and truth is not coincidental. The video suggests that in his 'shadow,' conflict with authorities and a struggle between truth and control emerge. It poses a ques

In [9]:
analysis_result_json

{'summary': "The video discusses Călin Georgescu, stating he was born under the archetype of 'The Emperor,' which signifies a destiny requiring vision, power, and discipline, but also carries a karma of not abusing power and submitting ego to truth. The speaker claims that Georgescu's focus on order and truth is not coincidental. The video suggests that in his 'shadow,' conflict with authorities and a struggle between truth and control emerge. It poses a question to the viewer about whether Georgescu is a leader or a fighter against the system and invites them to visit the speaker's website to discover their own birth day karma.",
 'identified_subjects': [{'subject': 'calin georgescu', 'stance': 0.8},
  {'subject': 'romania', 'stance': 0.7},
  {'subject': 'order', 'stance': 0.8},
  {'subject': 'truth', 'stance': 0.8},
  {'subject': 'power', 'stance': 0.5},
  {'subject': 'discipline', 'stance': 0.5},
  {'subject': 'authority', 'stance': -0.7},
  {'subject': 'control', 'stance': -0.7},
 

In [10]:
analysis_result_json

{'summary': "The video discusses Călin Georgescu, stating he was born under the archetype of 'The Emperor,' which signifies a destiny requiring vision, power, and discipline, but also carries a karma of not abusing power and submitting ego to truth. The speaker claims that Georgescu's focus on order and truth is not coincidental. The video suggests that in his 'shadow,' conflict with authorities and a struggle between truth and control emerge. It poses a question to the viewer about whether Georgescu is a leader or a fighter against the system and invites them to visit the speaker's website to discover their own birth day karma.",
 'identified_subjects': [{'subject': 'calin georgescu', 'stance': 0.8},
  {'subject': 'romania', 'stance': 0.7},
  {'subject': 'order', 'stance': 0.8},
  {'subject': 'truth', 'stance': 0.8},
  {'subject': 'power', 'stance': 0.5},
  {'subject': 'discipline', 'stance': 0.5},
  {'subject': 'authority', 'stance': -0.7},
  {'subject': 'control', 'stance': -0.7},
 